In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001904B9145D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001904BA84A90>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Arul and I am a AI Engineer")])

AIMessage(content="Nice to meet you, Arul. Welcome to our conversation. AI Engineering is a fascinating field, and I'm sure you have a lot of interesting stories and experiences to share. What specific areas of AI Engineering are you interested in or have experience with? Are you working on any exciting projects right now?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 48, 'total_tokens': 110, 'completion_time': 0.10781502, 'completion_tokens_details': None, 'prompt_time': 0.003364998, 'prompt_tokens_details': None, 'queue_time': 0.056335151, 'total_time': 0.111180018}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b0340-9a92-7610-b6e7-3bce08e70c68-0', usage_metadata={'input_tokens': 48, 'output_tokens': 62, 'total_tokens': 110})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Arul and I am a AI Engineer"),
        AIMessage(content="Hello Arul! It's nice to meet you. \n\nAs an AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Your name is Arul, and you're an AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 117, 'total_tokens': 131, 'completion_time': 0.017238999, 'completion_tokens_details': None, 'prompt_time': 0.006828134, 'prompt_tokens_details': None, 'queue_time': 0.048969666, 'total_time': 0.024067133}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b0340-bc40-7c00-a48f-ccc3274ff27a-0', usage_metadata={'input_tokens': 117, 'output_tokens': 14, 'total_tokens': 131})

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Arul and I am a AI Engineer")],
    config=config
)

In [8]:
response.content

"Nice to meet you, Arul. As an AI Engineer, you must be working on creating intelligent systems and developing innovative solutions using artificial intelligence and machine learning. It's a fascinating field that's rapidly evolving and transforming various industries.\n\nWhat specific areas of AI do you specialize in, Arul? Are you working on natural language processing, computer vision, robotics, or perhaps deep learning?"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Arul.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 140, 'total_tokens': 147, 'completion_time': 0.003835906, 'completion_tokens_details': None, 'prompt_time': 0.009072443, 'prompt_tokens_details': None, 'queue_time': 0.052055567, 'total_time': 0.012908349}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b0341-d420-7051-9591-9a8a2c706a33-0', usage_metadata={'input_tokens': 140, 'output_tokens': 7, 'total_tokens': 147})

In [10]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"I don't have any information about your name. I'm a large language model, I don't have the ability to retain information about individual users or remember previous conversations. Each time you interact with me, it's a new conversation. If you'd like to share your name with me, I'd be happy to chat with you!"

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Arul")],
    config=config1
)
response.content

"Nice to meet you, Arul. It was a pleasure to learn your name. Is there anything in particular you'd like to talk about or any questions you have that I can help with?"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Arul.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Arul")]})

AIMessage(content='Hello Arul, nice to meet you. Is there something I can help you with today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 57, 'total_tokens': 77, 'completion_time': 0.029002876, 'completion_tokens_details': None, 'prompt_time': 0.002818901, 'prompt_tokens_details': None, 'queue_time': 0.049543529, 'total_time': 0.031821777}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b0342-819a-7e91-8e39-f57a4323da85-0', usage_metadata={'input_tokens': 57, 'output_tokens': 20, 'total_tokens': 77})

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Arul")],
    config=config
)

response

AIMessage(content="Nice to meet you, Arul. I'm here to help with any questions or information you need. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 57, 'total_tokens': 87, 'completion_time': 0.039341472, 'completion_tokens_details': None, 'prompt_time': 0.002836702, 'prompt_tokens_details': None, 'queue_time': 0.054571978, 'total_time': 0.042178174}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b0342-968b-7500-b591-9ece3b701804-0', usage_metadata={'input_tokens': 57, 'output_tokens': 30, 'total_tokens': 87})

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Arul.'

In [18]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [19]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Arul")],"language":"German"})
response.content

'Hallo Arul, wie kann ich Ihnen heute helfen? (Hallo Arul, wie kann ich Ihnen heute helfen?)'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [20]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [23]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Arul")],"language":"tamil"},
    config=config
)
repsonse.content

'வணக்கம் அருள்! நான் உங்களுக்கு உதவியாக இருக்க இங்கு உள்ளேன். உங்களுக்கு என்ன உதவி தேவைப்படுகிறது?'

In [24]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "tamil"},
    config=config,
)

In [25]:
response.content

'உங்கள் பெயர் அருள்.'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.messages.utils import trim_messages, count_tokens_approximately

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=count_tokens_approximately,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmed = trimmer.invoke(messages)
print(trimmed)


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}), HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}), AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}), HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}), AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]


In [51]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [52]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"Unfortunately, I'm a large language model, I don't have any information about your personal preferences. However, I can suggest some popular ice cream flavors that people often enjoy:\n\n1. Vanilla\n2. Chocolate\n3. Strawberry\n4. Cookies and Cream\n5. Mint Chocolate Chip\n\nIf you'd like to share your favorite ice cream flavor with me, I'd be happy to chat about it!"

In [53]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

"You didn't ask a math problem. Our conversation just started, and I'm ready to help with any questions or problems you may have."

In [54]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [55]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I don't have any information about your name. This is the beginning of our conversation, and I'm happy to chat with you. If you'd like to share your name, I'd be happy to learn it!"